For this model, we used mostly the ideas from the https://www.kaggle.com/mobassir/mining-covid-19-scientific-papers/comments notebook, but we adapted the code to our particular case. 

For the pre-processing, go here: https://www.kaggle.com/patriciacanton/proyect-pre-processing

This model explores one of the possibilities. The others can be found in my main page: https://www.kaggle.com/patriciacanton

In [ ]:
#First we are importing the libraries we are going smeelingly use. Later on, we will import other libraries if they are necessary

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import json
import re
import string
import networkx as nx
import gc

from IPython.utils import io
#with io.capture_output() as captured:

 # !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_lg-0.2.4.tar.gz

from pprint import pprint
from copy import deepcopy

from collections import  Counter
from tqdm.notebook import tqdm
#LANGUAGE DETECTION
!pip install langdetect
import langdetect
from langdetect import detect
from langdetect import DetectorFactory
#NATURAL LANGUAGE PROCESSING

#1. NLTK


!pip install nltk
import nltk as nlp
#nltk.download()
from nltk import tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,PorterStemmer

#from rake_nltk import Rake
#2. SPACY

#!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_lg-0.2.4.tar.gz
#!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_ner_bionlp13cg_md-0.2.4.tar.gz 
!pip install scispacy
import scispacy
#import en_ner_bionlp13cg_md
#import en_core_sci_lg

from spacy import displacy
from scispacy.abbreviation import AbbreviationDetector
from scispacy.umls_linking import UmlsEntityLinker

#STOPWORDS IN ENGLISH SPACY
from spacy.lang.en.stop_words import STOP_WORDS

#TOKENIZER
from spacy.tokens import Span

#GENSIM

import gensim
# SPACY MODEL FOR LDA visualization

import pyLDAvis
import pyLDAvis.gensim

#TENSOR FLOWS
import tensorflow_hub as hub
import tensorflow as tf

#MATCHER
from spacy.matcher import Matcher 

#SKLEARN

from sklearn.metrics.pairwise import cosine_similarity

#from sklearn.cluster import DBSCAN
#os.listdir('../input/CORD-19-research-challenge/')
#with open('../input/CORD-19-research-challenge/metadata.readme', 'r') as f:
#   data = f.read()
#    print(data)

In [ ]:
os.listdir( '/kaggle/input/all-cleaned')


*******************


WE ARE GOING TO FOCCUS ON BIORXIV subset and on the abstract part

In [ ]:
biorxiv_clean = pd.read_csv('/kaggle/input/all-cleaned/biorxiv_clean.csv')


In [ ]:
biorxiv_clean.head(2)

checking everythin is ok with the first paper


In [ ]:
biorxiv_clean.text[0]

**************************************


MINING COVID 

This part of the code is directly developped following the mod

The first step would be to concatenate all the text contained in the articles Then split the text into individual sentences In the next step, we will find vector representation (word embeddings) for each and every sentence Similarities between sentence vectors are then calculated and stored in a matrix The similarity matrix is then converted into a graph, with sentences as vertices and similarity scores as edges, for sentence rank calculation Finally, a certain number of top-ranked sentences form the final summary

CONVERT ABSTRACT TO LIST

In [ ]:
df = biorxiv_clean
df = df.abstract.dropna()
data = df.values.tolist()

Find similar research papers using universalsentenceencoderlarge4

In [ ]:
%%time
#ref : https://gist.github.com/gaurav5430/8d7810495ec3f914ffb151458f352c60

'''import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity
def prepare_similarity(vectors):
    similarity=cosine_similarity(vectors)
    return similarity

def get_top_similar(sentence, sentence_list, similarity_matrix, topN):
    # find the index of sentence in list
    index = sentence_list.index(sentence)
    # get the corresponding row in similarity matrix
    similarity_row = np.array(similarity_matrix[index, :])
    # get the indices of top similar
    indices = similarity_row.argsort()[-topN:][::-1]
    return [sentence_list[i] for i in indices]


meta=pd.read_csv("../input/CORD-19-research-challenge/metadata.csv")
module_url = "../input/universalsentenceencoderlarge4" 
embed = hub.load(module_url)


# Creating an empty Dataframe with column names only
simsentence = pd.DataFrame()

titles=meta['title'].fillna("Unknown")
embed_vectors=embed(titles[:5000].values)['outputs'].numpy()
sentence_list=titles.values.tolist()
for i in range(5):

    sentences=titles.iloc[i]
    #print(">>>>>>>>>>>>Using title Find similar research papers for :",sentences, "<<<<<<<<<<<<")

    similarity_matrix=prepare_similarity(embed_vectors)
    similar=get_top_similar(sentences,sentence_list,similarity_matrix,6)
    for sentence in similar:
        #print(sentence)
        simsentence = simsentence.append({'sentence': sentences, 'similar': sentence}, ignore_index=True)
        #print("\n") '''

Now let's save this new dataframe as csv file for possible further research

In [ ]:
simsentence = pd.read_csv('../input/simsentence/simsentence.csv')

The function below converts sentences to words using gensim

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  

data_words = list(sent_to_words(data))

print(data_words[:1])

Build the bigram and trigram models using gensim

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=20) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=20)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[1]]])

Define functions for stopwords, bigrams, trigrams and lemmatization

We go and reach for the stopwords previously defined stopword, from https://www.kaggle.com/patriciacanton/proyect-pre-processing

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

stop_words.extend(['background', 'methods', 'introduction', 'conclusions', 'results', 
                   'purpose', 'materials', 'discussions','methodology','result analysis',
                   'doi', 'preprint', 'copyright', 'peer', 'reviewed', 'org', 'https', 'et', 'al', 'author', 'figure', 
                   'rights', 'reserved', 'permission', 'used', 'using', 'biorxiv', 'medrxiv', 'license', 'fig', 'fig.', 
                   'al.', 'Elsevier', 'PMC', 'CZI', 'www'])

In [ ]:
#https://github.com/cjriggio/classifying_medical_innovation
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[1])
print(data_lemmatized[:1])

Create Dictionary,Corpus and Document Frequency

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

Human readable format of corpus (term-frequency)

In [ ]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

Topic modeling with LDA, in order to have a first sight at the most common words


In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True
                                        )
#pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
%%time
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

Save Topics as html format

In [ ]:
pyLDAvis.save_html(vis, './l
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

Wordcloud of Top N words in each topic

In [ ]:
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  

cloud = WordCloud(stopwords=stop_words,
                  background_color='white',
                  width=2500,
                  height=1800,
                  max_words=100,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = lda_model.show_topics(formatted=False, 
                               num_words=30)

fig, axes = plt.subplots(5, 1, figsize=(10,20), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=500)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i + 1), fontdict=dict(size=16))
    plt.gca().axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()

Topic Distribution Plot

In [ ]:
from matplotlib.ticker import FuncFormatter

# Plot

fig, ax1  = plt.subplots(1, figsize=(10, 10))

# Topic Distribution by Dominant Topics
ax1.bar(x='Dominant_Topic', height='count', data=df_dominant_topic_in_each_doc, width=.5, color='firebrick')
ax1.set_xticks(range(df_dominant_topic_in_each_doc.Dominant_Topic.unique().__len__()))
tick_formatter = FuncFormatter(lambda x, pos: 'Topic ' + str(x + 1)+ ':\n' + df_top3words.loc[df_top3words.topic_id==x, 'words'].values[0])
ax1.xaxis.set_major_formatter(tick_formatter)
ax1.set_title('Number of Documents by Dominant Topic', fontdict=dict(size=10))
ax1.set_ylabel('Number of Documents')
ax1.set_ylim(0, 2000)

plt.show()
df_dominant_topic_in_each_doc
fig, ax2  = plt.subplots(1, figsize=(10, 10))
# Topic Distribution by Topic Weights
ax2.bar(x='index', height='count', data=df_topic_weightage_by_doc, width=.5, color='steelblue')
ax2.set_xticks(range(df_topic_weightage_by_doc.index.unique().__len__()))
ax2.xaxis.set_major_formatter(tick_formatter)
ax2.set_title('Number of Documents by Topic Weightage', fontdict=dict(size=10))

plt.show()

Get topic weights and dominant topics

In [ ]:
from sklearn.manifold import TSNE
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook

# Get topic weights
topic_weights = []
for i, row_list in enumerate(lda_model[corpus]):
    topic_weights.append([w for i, w in row_list[0]])

# Array of topic weights    
arr = pd.DataFrame(topic_weights).fillna(0).values

# Keep the well separated points (optional)
arr = arr[np.amax(arr, axis=1) > 0.35]

# Dominant topic number in each doc
topic_num = np.argmax(arr, axis=1)

# tSNE Dimension Reduction
tsne_model = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')
tsne_lda = tsne_model.fit_transform(arr)

# Plot the Topic Clusters using Bokeh
output_notebook()
n_topics = 5
mycolors = np.array([color for name, color in mcolors.TABLEAU_COLORS.items()])


plot = figure(title="t-SNE Clustering of {} LDA Topics".format(n_topics), 
              plot_width=800, plot_height=600)
plot.scatter(x=tsne_lda[:,0], y=tsne_lda[:,1], color=mycolors[topic_num])
show(plot)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

    # # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # # Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

    # # Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

In [ ]:
vectorizer = CountVectorizer(analyzer='word', min_df=10,                        # minimum reqd occurences of a word 
                              stop_words='english',             # remove stop words
                              lowercase=True,                   # convert all words to lowercase
                              token_pattern='[a-zA-Z0-9]{3,}'  # num chars > 3
                              # max_features=50000,             # max number of uniq words
                             )
data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [ ]:
## Define Search Param for GridSearch
%%time
search_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation(n_jobs=-1)

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)


# Do the Grid Search
model.fit(data_vectorized)

Best Model

In [ ]:
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))
categories = list(df.Dominant_Topic.unique())
categories
X_embedded[:,1].shape
# sns settings
sns.set(rc={'figure.figsize':(10,10)})

# colors
palette = sns.color_palette("bright", len(set(y_pred)))

# plot
sns.scatterplot(X_embedded[:,0], X_embedded[:,1], hue=y_pred, legend='full', palette=palette)
plt.title("t-SNE Covid-19 Articles - Clustered")
# plt.savefig("plots/t-sne_covid19_label.png")
plt.show()

In [ ]:
#CAMBIAR NON-COMMON POR EL MÍO
type(clean_noncomm_use.abstract.dropna().tolist())
from gensim.summarization.summarizer import summarize
summarize(clean_noncomm_use.abstract.dropna().to_string())
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re
#ref : https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/
from nltk.tokenize import sent_tokenize
sentences = []
for s in clean_noncomm_use.abstract.dropna():
    sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x] # flatten list
sentences[:3]

Download GloVe Word Embeddings

In [ ]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()
len(word_embeddings)
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

REMOVE STOPWORDS

In [ ]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
Vector Representation of Sentences

# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

Similarity Matrix preparation

In [ ]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])
import torch
torch.cuda.is_available()
%%time
from sklearn.metrics.pairwise import cosine_similarity
for i in range(1000):
    for j in range(1000):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
            #print(sim_mat[i][j])

Applying PageRank Algorithm

In [ ]:
%%time
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

Summary Extraction : Extract top 50 sentences as the summary

In [ ]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
for i in range(50):
    print(ranked_sentences[i][1])
    print('\n\n')

BART

In [ ]:
import os
import json
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
BART_PATH = 'bart-large'
bart_model = BartForConditionalGeneration.from_pretrained(BART_PATH, output_past=True)
bart_tokenizer = BartTokenizer.from_pretrained(BART_PATH)
def bart_summarize(input_text, num_beams=4, num_words=80):
    #input_text = str(input_text)
    input_text = ' '.join(input_text.split())
    input_tokenized = bart_tokenizer.encode(input_text, return_tensors='pt')
    summary_ids = bart_model.generate(input_tokenized,
                                      num_beams=int(num_beams),
                                      no_repeat_ngram_size=3,
                                      length_penalty=2.0,
                                      min_length=100,
                                      max_length=int(num_words),
                                      early_stopping=True)
    output = [bart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    return output[0]
df = clean_pmc
df = df.abstract.dropna()
abstracts = df.values.tolist()

len(abstracts)
#summarizing
for i in range(20):
    try:
        print('paper  ',i + 1, " : \n" )
        print(bart_summarize(abstracts[i]))
        print('............................................................................\n\n\n\n')
    except:
        print('paper ',i+1 ," has LONG ABSTRACT\n\n")

Summarization Task using T5 model

In [ ]:
df1 = df1.abstract.dropna()
df1abstracts = df.values.tolist()

len(df1abstracts)
T5_PATH = 't5-base'
t5_model = T5ForConditionalGeneration.from_pretrained(T5_PATH, output_past=True)
t5_tokenizer = T5Tokenizer.from_pretrained(T5_PATH)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def t5_summarize(input_text, num_beams=4, num_words=80):
    #input_text = str(input_text).replace('\n', '')
    input_text = ' '.join(input_text.split())
    input_tokenized = t5_tokenizer.encode(input_text, return_tensors="pt").to(device)
    summary_task = torch.tensor([[21603, 10]]).to(device)
    input_tokenized = torch.cat([summary_task, input_tokenized], dim=-1).to(device)
    summary_ids = t5_model.generate(input_tokenized,
                                    num_beams=int(num_beams),
                                    no_repeat_ngram_size=3,
                                    length_penalty=2.0,
                                    min_length=30,
                                    max_length=int(num_words),
                                    early_stopping=True)
    output = [t5_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    return output[0]

summariing first 20 papers abstract of bioarvix



In [ ]:
%%time
for i in range(20):
    try:
        print('BioArvix paper  ',i + 1, " : \n" )
        print(t5_summarize(df1abstracts[i]))
        print('............................................................................\n\n\n\n')
    except:
        print('paper ',i+1 ," has LONG ABSTRACT\n\n")
simsentence.head(5)
"""
This is a simple application for sentence embeddings: semantic search
We have a corpus with various sentences. Then, for a given query sentence,
we want to find the most similar sentence in this corpus.
This script outputs for various queries the top 5 most similar sentences in the corpus.
"""
# taken from : https://github.com/UKPLab/sentence-transformers/blob/master/examples/application_semantic_search.py
from sentence_transformers import SentenceTransformer
import scipy.spatial

embedder = SentenceTransformer('bert-base-nli-mean-tokens')

# Corpus with example sentences
corpus = df.values.tolist()
corpus_embeddings = embedder.encode(corpus)

# Query sentences:
queries = ['Range of incubation periods for the disease in humans','risk factors of covid-19','cure for covid-19', 'antiviral covid-19 success treatment','Does smoking or pre-existing pulmonary disease increase risk of COVID-19?', 'risk of fatality among symptomatic hospitalized patients']
query_embeddings = embedder.encode(queries)

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
closest_n = 5
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx, distance in results[0:closest_n]:
        print(corpus[idx].strip(), "(Score: %.4f)" % (1-distance))